## You can put your name here

* Name:
* Net ID:

### Note: This template is used to save your time. If there are some typos in this file, please **refer to the original PDF file of problems.**

Problem Set 4

CS 6375

Due: 4/21/2022 by 11:59pm

Note: all answers should be accompanied by explanations and relevant code for full credit. All
code (Python or MATLAB only) should be turned in with your answers to the following questions. Python is preferred.  
Late homeworks will not be accepted. 

The following code help you download the data files needed for the problems below. After running code, these files will appear in the current folder.

In [416]:
#Grab Data
import numpy as np
import pandas as pd

import sys, os
import math
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Downloading data files for you.
# Remember to run this cell!!

!wget -nc https://personal.utdallas.edu/~yangxiao.lu/22SP_CS6375/leaf.data
!wget -nc https://personal.utdallas.edu/~yangxiao.lu/22SP_CS6375/prostate_GE.data


File 'leaf.data' already there; not retrieving.

File 'prostate_GE.data' already there; not retrieving.



#$\color{blue}{\text{Problem 1:}} \text{ PCA and Feature Selection} \color{red}{\text{(50 pts)}}$#

In this problem, we will explore ways that we can use PCA for the problem of generating or selecting "good" features.

##1.1 SVMs and PCA (25 pts)


Consider the prostate GE data set (attached to this problem set). Each row corresponds to a single
data point and the final column in each row is the class label (1 or 2 here). You should use the
first 80 data points for training and the remaining 10% for testing.

* Perform PCA on the training data to reduce the dimensionality of the data set (ignoring the
class labels for the moment). What are the top six eigenvalues of the data covariance matrix?
Looking at the eigenvalues, how would you recommend picking k for this data set?


In [417]:

inputData =  pd.read_csv('prostate_GE.data', sep=",", header=None)
trainData = inputData[:80]
testData = inputData[80:]

pca = PCA()
X = trainData.drop(columns=[5966])

#print(Y)
XTrans = pca.fit_transform(X)
#print(XTrans)
eig_vals = pca.explained_variance_
contrib_func = np.vectorize(lambda x: x / np.sum(eig_vals))
var_contrib = contrib_func(eig_vals)
print("Contributions of each eigen value: ",var_contrib, "\n Contribution of first 25 eigen values: ", np.sum(var_contrib[:25]))

print("Top 6 eigen values: ", eig_vals[:6], "\n Contribution of first 6 eigen values: ", np.sum(var_contrib[:6]))

Contributions of each eigen value:  [5.70393485e-01 4.53756540e-02 4.11866780e-02 2.06098695e-02
 1.64962389e-02 1.61281384e-02 1.32959658e-02 1.16461022e-02
 1.10719621e-02 1.03953876e-02 9.31934774e-03 8.78355201e-03
 8.16483103e-03 7.70609411e-03 7.45004888e-03 7.15805429e-03
 6.92469672e-03 6.73605211e-03 6.47834449e-03 6.29482718e-03
 6.19111620e-03 5.86850209e-03 5.74144359e-03 5.61275323e-03
 5.38048638e-03 5.20629269e-03 4.96566507e-03 4.89605081e-03
 4.71313577e-03 4.69398191e-03 4.54133824e-03 4.33303641e-03
 4.07871200e-03 3.89868367e-03 3.81493159e-03 3.72359111e-03
 3.61028952e-03 3.57723339e-03 3.43525956e-03 3.31371713e-03
 3.26609401e-03 3.15966104e-03 3.06905152e-03 2.97378529e-03
 2.89043898e-03 2.80645551e-03 2.76688678e-03 2.70464152e-03
 2.60200913e-03 2.50907787e-03 2.47191191e-03 2.44241306e-03
 2.33491010e-03 2.27630158e-03 2.23719427e-03 2.08708704e-03
 2.01460970e-03 2.00108819e-03 1.92522312e-03 1.87384167e-03
 1.80386576e-03 1.76851632e-03 1.73001141e-03 1.6

# From the contribution, The top 6 eigenvalues contributes to 71% of the information. k=25 will retain 86% of the original information, so k=25 is recomended.

* For each k ∈ {1, 2, 3, 4, 5, 6}, project the training data into the best k dimensional subspace
(with respect to the Frobenius norm) and use the SVM with slack formulation to learn a
classifier for each c ∈ {1, 10, 100, 1000}. As data is limited, this process should be done with
10-fold cross-validation. Report the average error of the learned classifier on the held out
validation data for each k and c pair.

In [418]:
k = [1,2,3,4,5,6]
c = [1,10,100,1000]



In [419]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

for kVal in k:
    acc=[]
    pca = PCA(n_components=kVal)
    XTrans = pca.fit_transform(X)
    XTest_Trans = pca.fit_transform(XTest)
    Y = trainData[5966].values
    Y.reshape(-1,1)
    XTest = testData.drop(columns=[5966])
    YTest = testData[5966].values

    for cSelection in c:
        acc=[]
        clf = make_pipeline(StandardScaler(), SVC(C=cSelection, kernel='poly', gamma='auto'))
        clf.fit(XTrans, Y)
        YPred = clf.predict(XTest_Trans)

        res = sum([int(YTest[i] == YPred[i]) for i in range(YTest.shape[0])])
        acc.append(res/YTest.shape[0])
        print("With k = ", kVal, "and C =", cSelection, ': Hit = ', res, 'in a total of ',YTest.shape[0])
    print('Average Accuracy of all c is:', np.array(acc).mean())

With k =  1 and C = 1 : Hit =  11 in a total of  22
With k =  1 and C = 10 : Hit =  11 in a total of  22
With k =  1 and C = 100 : Hit =  11 in a total of  22
With k =  1 and C = 1000 : Hit =  11 in a total of  22
Average Accuracy of all c is: 0.5
With k =  2 and C = 1 : Hit =  9 in a total of  22
With k =  2 and C = 10 : Hit =  10 in a total of  22
With k =  2 and C = 100 : Hit =  10 in a total of  22
With k =  2 and C = 1000 : Hit =  10 in a total of  22
Average Accuracy of all c is: 0.45454545454545453
With k =  3 and C = 1 : Hit =  14 in a total of  22
With k =  3 and C = 10 : Hit =  16 in a total of  22
With k =  3 and C = 100 : Hit =  14 in a total of  22
With k =  3 and C = 1000 : Hit =  13 in a total of  22
Average Accuracy of all c is: 0.5909090909090909
With k =  4 and C = 1 : Hit =  14 in a total of  22
With k =  4 and C = 10 : Hit =  15 in a total of  22
With k =  4 and C = 100 : Hit =  15 in a total of  22
With k =  4 and C = 1000 : Hit =  13 in a total of  22
Average Accu

* What is the performance you achieve on the test set via the proper hyperparameter selection
procedure above?

With k = 3 and c =10, The performance reaches the highest (15 correct out of 22). 
k = 3 is the best k value due to its eliminatio of more values but reaching a relatively good performance.

* Now suppose that we don’t do proper hyperparameter selection. What is the best performance
that you can achieve on the test set if you tune the hyperparameters using the test set instead
of the validation set?

In [420]:
for kVal in range(7,15):
    acc=[]
    pca = PCA(n_components=kVal)
    XTrans = pca.fit_transform(X)
    XTest_Trans = pca.fit_transform(XTest)
    Y = trainData[5966].values
    Y.reshape(-1,1)
    XTest = testData.drop(columns=[5966])
    YTest = testData[5966].values

    for cSelection in c:
        acc=[]
        clf = make_pipeline(StandardScaler(), SVC(C=cSelection, kernel='poly', gamma='auto'))
        clf.fit(XTrans, Y)
        YPred = clf.predict(XTest_Trans)

        res = sum([int(YTest[i] == YPred[i]) for i in range(YTest.shape[0])])
        acc.append(res/YTest.shape[0])
        print("With k = ", kVal, "and C =", cSelection, ': Hit = ', res, 'in a total of ',YTest.shape[0])
    print('Average Accuracy of all c is:', np.array(acc).mean())

With k =  7 and C = 1 : Hit =  13 in a total of  22
With k =  7 and C = 10 : Hit =  13 in a total of  22
With k =  7 and C = 100 : Hit =  13 in a total of  22
With k =  7 and C = 1000 : Hit =  13 in a total of  22
Average Accuracy of all c is: 0.5909090909090909
With k =  8 and C = 1 : Hit =  13 in a total of  22
With k =  8 and C = 10 : Hit =  12 in a total of  22
With k =  8 and C = 100 : Hit =  13 in a total of  22
With k =  8 and C = 1000 : Hit =  13 in a total of  22
Average Accuracy of all c is: 0.5909090909090909
With k =  9 and C = 1 : Hit =  14 in a total of  22
With k =  9 and C = 10 : Hit =  11 in a total of  22
With k =  9 and C = 100 : Hit =  12 in a total of  22
With k =  9 and C = 1000 : Hit =  13 in a total of  22
Average Accuracy of all c is: 0.5909090909090909
With k =  10 and C = 1 : Hit =  14 in a total of  22
With k =  10 and C = 10 : Hit =  14 in a total of  22
With k =  10 and C = 100 : Hit =  15 in a total of  22
With k =  10 and C = 1000 : Hit =  15 in a total 

As can be seen, from the testing set, k = 10 produces the best accuracy at 68%.

## 1.2 PCA for Feature Selection (25 pts)

If we performed PCA directly on the training data as we did in the first part of this question, we
would generate new features that are linear combinations of our original features. If instead, we
wanted to find a subset of our current features that were good for classification, we could still use
PCA, but we would need to be more clever about it. The primary idea in this approach is to select features from the data that are good at explaining as much of the variance as possible. To do this,
we can use the results of PCA as a guide. Implement the following algorithm for a given k and s:
1.   Compute the top $k$ eigenvalues and eigenvector of the covariance matrix corresponding to the data matrix omitting the labels (recall that the columns of the data matrix are the input data points). Denote the top $k$ eigenvectors as $v^{(1)}, ... , v^{(k)}$.

2.   Define $\pi_j = \frac{1}{k}\sum_{i=1}^k v_j^{(i)^2}$.

3.   Sample $s$ features independently from the probability distribution defined by $\pi$.

*  Why does $\pi$ define a probability distribution?

*  Again, using the prostate_GE data set and same procedure as above, for each $k\in\{1,10,20,40,80,160\}$ with $s=\lfloor k\log k\rfloor$, report the average test error of the SVM with slack classifier over $20$ experiments. For each experiment use only the $s$ selected features (note that there may be some duplicates, so only include each feature once). Use the same hyperparameter search for $c$ as in part 1.

* Does this provide a reasonable alternative to the SVM with slack formulation without feature
selection on this data set? What are the pros and cons of this approach?

#$\color{blue}{\text{Problem 2:}} \text{ Working with k-means} \color{red}{\text{(50 pts)}}$#

For this problem, you will use the leaf.data file provided with this problem set. This data set
was generated from the UCI Leaf Data Set (follow the link for information about the format of the
data). The class labels are still in the data set and should be used for evaluation only (i.e., don’t
use them in the clustering procedure), but the specimen number has been removed. You should
preprocess the data so that the non-label attributes have mean zero and variance one.

## 2.1 Train a k-means classifier for each $k\in\{10, 15, 20, 25, 30\}$ starting from twenty different random initializations (sample uniformly from $[−3, 3]$ for each attribute) for each $k$. Report the
mean and variance of the value of the $k$-means objective obtained for each $k$.

In [421]:
N_DATA_AMOUNT=340
RV=0

In [422]:
col_names = ['y','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13','x14']
trainData =  pd.read_csv('leaf.data', sep="," , names = col_names)
#TrainDataNew contains preprocessed Data
trainDataNew = trainData.drop(columns=['y'])
#Store the preAdjusted Mean and Standard Deviation
trainMean = trainDataNew.mean()
trainSTD = trainDataNew.std()
#print("trainMean is: \n", trainMean)
#print("trainSTD is \n", trainSTD)
trainDataNew = (trainDataNew-trainMean)/trainSTD

In [423]:
kVal=[10,15,20,25,30]


for k in kVal:
    centersSelected = []
    for iterations in range(k):
        centersFetSelected = []
        RV=np.random.uniform(low=-3,high=3,size=20)
        RV=((RV%N_DATA_AMOUNT))
        for idx in range(20):
            RV[idx] = int(RV[idx])

        rowsSelected = trainDataNew.iloc[RV,0:]
        for idx in range(14):
            centersFetSelected.append(rowsSelected.mean()[idx])
        centersSelected.append(centersFetSelected)
        
    centersSelected = np.array(centersSelected)
    kmeans = KMeans(n_clusters = k, init=centersSelected, random_state=1, n_init=1)
    kmeans.fit(trainDataNew)
    centers = kmeans.cluster_centers_
    #print(centers)
    centers = pd.DataFrame(data=centers[:,:], columns=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13','x14'])
    centersMean = centers.mean()
    centersSTD = centers.std()
    print("K = ", k)
    print(centers)
    print("K = ", k, "center's Mean is: \n", centersMean)
    print("K = ", k, "center's Variance is \n", centersSTD**2)

K =  10
         x1        x2        x3        x4        x5        x6        x7  \
0 -0.265226 -0.304245 -0.489363  0.378863  0.392753  0.339310 -0.228871   
1 -1.406347 -0.518779  0.859338 -3.186378 -2.894113 -1.727637  2.439976   
2  0.861924  0.025556  0.514247  0.315729  0.319266 -0.085595 -0.265616   
3 -1.384623 -0.515278  0.671503 -1.509767 -1.623472 -1.457132  1.111967   
4  1.303199  2.587141  1.978144 -1.515707 -0.220531 -1.674218  3.297187   
5 -0.884856 -0.502723  0.580235 -1.601683 -2.330870 -1.301411  1.996235   
6  0.191413 -0.270899 -0.441537  0.522929  0.437678  0.752471 -0.577058   
7  0.179777 -0.271835  0.264699 -0.515079 -0.480975 -0.651743  0.793029   
8 -0.680756 -0.430448 -1.037842  0.446010  0.443593  0.782074 -0.538156   
9  1.307830  2.751569  1.914578  0.159189  0.239481 -1.479951 -0.137185   

         x8        x9       x10       x11       x12       x13       x14  
0 -0.311394  1.824712  1.615862  1.783753  1.542576  1.641771  1.484010  
1  2.538360 -0.591

K =  25
          x1        x2        x3        x4        x5        x6        x7  \
0  -1.547095 -0.514177  1.450643 -1.932118 -3.173462 -1.865126  1.410338   
1   1.320937  2.520463  2.117411 -1.895406 -0.655152 -1.776540  3.919267   
2   1.333659  3.681659  2.206376 -2.042127  0.000583 -1.931184  4.190168   
3  -0.937584 -0.513129 -0.090961 -1.185686 -0.543313 -1.122873  1.004847   
4   1.338651  6.386117  2.221971 -0.463696 -0.319636 -2.045913  0.925560   
5   0.513311 -0.191381 -0.130526  0.513677  0.449031  0.636196 -0.569464   
6   0.810505 -0.020173  0.366577  0.543768  0.457540  0.219757 -0.524113   
7  -1.299587 -0.513903  0.776218 -2.825467 -2.774808 -1.635206  2.334947   
8   1.327826  3.107410  2.135792 -1.312141 -0.190036 -1.830578  2.608182   
9  -1.212534 -0.481951 -0.747920  0.098241  0.290291  0.223951  0.362681   
10 -0.746447 -0.451854 -1.031112  0.337516  0.430906  0.594873 -0.475001   
11  1.334283  4.761672  2.148311 -0.109629 -0.014687 -1.864547 -0.147625   
12 -

## 2.2 Random initializations can easily get stuck in suboptimal clusterings. An improvement of the
k-means algorithm, known as k-means++, instead chooses an initialization as follows:

  (a) Choose a data point uniformly at random to be the first center.

(b) Repeat the following until k centers have been selected:  
i. For each data point x compute the distance between x and the nearest cluster center
in the current set of centers. Denote this distance as dx.

ii. Sample a training data point at random from the distribution $p$ such that $p(x) ∝ d^2_x $
Add the sampled point to the current set of centers.


Repeat the first experiment using this initialization to pick the initial cluster centers for
k-means. Does this procedure result in an improvement? Explain.


In [472]:
RowNumber = []
RowNumber.append(np.random.uniform(0, N_DATA_AMOUNT))
RowNumber[0] = int(FirstCenterRowNumber)
print(RowNumber[0])

for k in kVal:
    RowSelected = trainDataNew.iloc[RowNumber[0],0:]
    distances=[]
    for idx in range(N_DATA_AMOUNT):
        distances.append(np.linalg.norm(firstRowSelected - trainDataNew.iloc[idx,0:]))
    #Make largest distance point the next point will fit d^2x
    #distances= np.array(distances)
    RowNumber.append(np.where(distances == np.amax(distances))[0][0])



36


[36, 277]